### As I started this project late, this is atempt #1 to do the entire project till cloud deployment. The focus will be on deployment. In model 0 and 1 I hope to do more EDA and custome Piplines
### This files workflow 
1. Simple base model 0
2. Null values replaced with mean 
3. Categorical variables to numerical 
4. Split Data and train the model 
**5. Serialize Using Pickle**
**6. Deploy using flask locally and on AWS VM**

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns  
from sklearn.metrics import f1_score

In [4]:
df = pd.read_csv("data.csv")
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
df = df.drop(columns = ['Loan_ID', 'Dependents'])
df.head()

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
df.isnull().sum()

Gender               13
Married               3
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
# replace nulls with mean
df = df.fillna(df.mean().iloc[0])

In [88]:
df.isnull().sum()

Gender               0
Married              0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [8]:
# Change categorical into dummy variables
df['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
df['Married'].replace({'Yes':1,'No':0},inplace=True)
df['Gender'].replace({'Male':1,'Female':0},inplace=True)
df['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
df['Property_Area'].replace({'Urban':2,'Semiurban':1,'Rural':0},inplace=True)
df['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [9]:
df

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1.0,0.0,1,0.0,5849,0.0,5403.459283,360.0,1.0,2,1
1,1.0,1.0,1,0.0,4583,1508.0,128.000000,360.0,1.0,0,0
2,1.0,1.0,1,1.0,3000,0.0,66.000000,360.0,1.0,2,1
3,1.0,1.0,0,0.0,2583,2358.0,120.000000,360.0,1.0,2,1
4,1.0,0.0,1,0.0,6000,0.0,141.000000,360.0,1.0,2,1
...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0.0,1,0.0,2900,0.0,71.000000,360.0,1.0,0,1
610,1.0,1.0,1,0.0,4106,0.0,40.000000,180.0,1.0,0,1
611,1.0,1.0,1,0.0,8072,240.0,253.000000,360.0,1.0,2,1
612,1.0,1.0,1,0.0,7583,0.0,187.000000,360.0,1.0,2,1


In [10]:
# Split the data set into training and test data 
X = df.drop(columns='Loan_Status')
y = df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27, stratify=y)

In [92]:
# # Scale x
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)

# # PCA
# pca = PCA(n_components=3)
# pca.fit(X_train_scaled)
# X_train_pca = pca.transform(X_train_scaled)

In [23]:
# Run logistics regregression 
regressor = LogisticRegression()
regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_test)
acc = accuracy_score(y_test, y_pred)
evaluation = f1_score(y_test, y_pred)
print(f'F1_score: {evaluation} ')
print(f'Test set accuracy: {acc}')

F1_score: 0.7960199004975124 
Test set accuracy: 0.6666666666666666


In [12]:
# saving the model
import pickle

# saving the columns
model_columns = list(X.columns)
with open('model_columns.pkl','wb') as file:
    pickle.dump(model_columns, file)
    
    
pickle.dump(regressor, open('regressor.pkl', 'wb'))

In [101]:
#Json Data User input
# json_data = [{'Gender': 1, 'Married': 1, 'Education' :1, 'Self_Employed' :0, 'ApplicantIncome':5849,
#        'CoapplicantIncome' : 1508.0, 'LoanAmount': 50.0, 'Loan_Amount_Term':360, 'Credit_History':1.0,
#        'Property_Area':0 } ]

In [16]:
from sklearn.tree import DecisionTreeClassifier
Clf = DecisionTreeClassifier()
Clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [22]:
y_p = Clf.predict(X_test)
acc = accuracy_score(y_test, y_p)
evaluation = f1_score(y_test, y_p)
print(f'F1_score: {evaluation} ')
print(f'Test set accuracy: {acc}')

F1_score: 0.7790697674418604 
Test set accuracy: 0.6910569105691057
